![headerimage](https://github.com/Fundator/NVDB-Hackathon/blob/master/imgs/front.png?raw=true)

# Examples of retrieving data from several sources
This is a short tutorial on how to download your own data from nvdb. It is written in python, but using HTTP-requests in other languages should follow the same principles.

# NVDB

## Read the docs
The API documentation for NVDB can be found [here](https://www.vegvesen.no/nvdb/apidokumentasjon/)

## Python-wrapper for NVDB, pnvdb
Github user Acurus has made a python-wrapper for NVDB. It can be found [here](https://github.com/Acurus/pnvdb)

## Necessary packages

In [1]:
import requests
import pandas as pd
import json

## List all types of road objects (vegobjekter)

In [ ]:
url = "https://www.vegvesen.no/nvdb/api/v2/vegobjekter"
r = requests.get(url)
vegobjekter = json.loads(r.content)

In [ ]:
df = pd.DataFrame(vegobjekter)
df["id"] = df["href"].apply(lambda x: x.split("/")[-1])

In [ ]:
df

## Get list of ID's of specific road object type

List of object ids can be seen [here](https://datakatalogen.vegdata.no/)

In [ ]:
objektid = 25 #  Leskur
url = f"https://www.vegvesen.no/nvdb/api/v2/vegobjekter/{objektid}"
r = requests.get(url)
vegobjekt = json.loads(r.content)

In [ ]:
vegobjekt

## Pagination, get large amounts of data
nvdb API has a limit of 1000 returned values per request. However, it has a handy paginationfunction to download higher amount of content. If we inspect the metadata part of the latest request, we can see that it includes an url to the next content

In [ ]:
vegobjekt["metadata"]

Lets get the next 1000 objects

In [ ]:
r = requests.get(vegobjekt["metadata"]["neste"]["href"])
next_1000 = json.loads(r.content)

In [ ]:
next_1000

## Acessing a specific road object

In [ ]:
index = 0
url = next_1000["objekter"][index]["href"]
r = requests.get(url)
objekt = json.loads(r.content)

In [ ]:
objekt

## Specifying and restricting the search
NVDB-API has several parameters to specify where to get data, for example 

- Karutsnitt (rectangular map)
- Fylke / Kommune (Region / Municipality)
- Kontraktsområde

More information can be found on the [API-documentation page](https://www.vegvesen.no/nvdb/apidokumentasjon/#/)

### Example, get all toll booths in trondheim

In [ ]:
vegobjid = 45  #  Toll booths
kommuneid = 5001  #  Trondheim
url = f"https://www.vegvesen.no/nvdb/api/v2/vegobjekter/{vegobjid}?kommune={kommuneid}"
r = requests.get(url)
toll_booths_trd = json.loads(r.content)
toll_booths_trd

### Lets check one of the toll booths in trondheim

In [ ]:
index = 0
example_toll_booth_url = toll_booths_trd["objekter"][index]["href"]
r = requests.get(example_toll_booth_url)
json.loads(r.content)

## Geospatial data in NVDB
Road objects in NVDB is described mostly using UTM33N.

In [ ]:
geometri = objekt["geometri"]
geometri

##  <font color=green>Tips & tricks<font>

## Using geodaisy to parse WKT
For most WKT-types, with the exception of some geometry-types, like LINESTRING Z, geodaisy is one of the most lightweight libraries for parsing wkt

In [ ]:
import geodaisy.converters as convert 

In [ ]:
geo = convert.wkt_to_geo_interface(geometri["wkt"])
geo

In [ ]:
easting, northing = geo["coordinates"]

## Converting UTM to lat-long
For some applications like google maps api, lat-long is the only accepted geo-description. **UTM** is an easy-to-use library for python for converting  between UTM and lat-long. <font color=red>Some geometries may exceed the northing / easting ranges<font>

In [ ]:
import utm

In [ ]:
lat, lon = utm.to_latlon(easting, northing, zone_number=33, northern=True, strict=False)

In [ ]:
lat, lon

# Air Quality data from NILU
NILU offers historical and live air quality data from all of Norway. 

## List of components
Components are different parameters measured by the sensors, such as CO, NO2. All components avaliable can be found by the following request

In [ ]:
url = "https://api.nilu.no/lookup/components"
r = requests.get(url)
comp = json.loads(r.content)
pd.DataFrame(comp)

## Get the latest values for all components for all stations in Trondheim

In [ ]:
components = [i["component"] for i in comp]
compstr = ";".join(components)
url = f"https://api.nilu.no/aq/utd?areas=trondheim&components={compstr}"
r = requests.get(url)
val = json.loads(r.content)
pd.DataFrame(val)

In [ ]:
url = f"https://api.nilu.no/obs/utd?areas=trondheim&components={compstr}"
r = requests.get(url)
val = json.loads(r.content)
pd.DataFrame(val)

## Find all stations in Trondheim

In [ ]:
url = f"https://api.nilu.no/lookup/stations?area=trondheim"
r = requests.get(url)
val = json.loads(r.content)
stations = pd.DataFrame(val)
stations

In [ ]:
stations = stations["station"].tolist()
months = pd.date_range("2015-01-01", "2019-08-01", freq="m")
months = [str(m).split(" ")[0] for m in months]

## Get observations from last 3 years

In [ ]:
l = []
for station in stations:
    fromdate = months[0]
    for todate in months[1:]:
        url = f"https://api.nilu.no/obs/historical/{fromdate}/{todate}/{station}/?components={compstr}"
        r = requests.get(url)
        val = json.loads(r.content)
        l.append(val)
        fromdate = todate

In [ ]:
d = pd.DataFrame()
for i in range(len(l)):
    d = d.append(pd.DataFrame(l[i]))

In [ ]:
j =  []
columns = d.drop(columns="values").columns.tolist()
for i in range(d.shape[0]):
    leftside = pd.DataFrame(d.iloc[i]).T
    rightside = pd.DataFrame(d.iloc[i]["values"])
    for c in columns:
        rightside.loc[:,c] = leftside[c].values
    j.append(rightside)

In [ ]:
dj = pd.concat(j)
dj.to_csv("airquality_observations_2015_2019.csv")

## Get air quality index from last 3 years

In [ ]:
l = []
for station in stations:
    fromdate = months[0]
    for todate in months[1:]:
        url = f"https://api.nilu.no/aq/historical/{fromdate}/{todate}/{station}/?components={compstr}"
        r = requests.get(url)
        val = json.loads(r.content)
        l.append(val)
        fromdate = todate

In [ ]:
d = pd.DataFrame()
for i in range(len(l)):
    d = d.append(pd.DataFrame(l[i]))

In [ ]:
j =  []
columns = d.drop(columns="values").columns.tolist()
for i in range(d.shape[0]):
    leftside = pd.DataFrame(d.iloc[i]).T
    rightside = pd.DataFrame(d.iloc[i]["values"])
    for c in columns:
        rightside.loc[:,c] = leftside[c].values
    j.append(rightside)

In [ ]:
dj = pd.concat(j)
dj.to_csv("airquality_index_2015_2019.csv", index=False)

# Weather data from frost.met.no

The [frost API](https://frost.met.no/index2.html) provides free access to MET Norway's archive of historical weather and climate data

## [Examples](https://frost.met.no/ex_userquest) from docs

## Find station from location name

In [ ]:
# Obtained for free for Hackathon
client_id = "6d9ad2eb-f1dd-4164-8c16-451d98f49116"
client_secret = "6e43d276-c0d2-4ae1-91d9-cc20adf65324"

In [16]:
location = "Trondheim*" 
src_endpoint = 'https://frost.met.no/sources/v0.jsonld'
parameters = {
    'types': 'SensorSystem',
    'name': location,
}

r = requests.get(src_endpoint, parameters, auth=(client_id,''))
# Extract JSON data
json = r.json()
json

{'@context': 'https://frost.met.no/schema',
 '@type': 'SourceResponse',
 'apiVersion': 'v0',
 'license': 'https://creativecommons.org/licenses/by/3.0/no/',
 'createdAt': '2019-06-24T05:19:25Z',
 'queryTime': 2.751,
 'currentItemCount': 5,
 'itemsPerPage': 5,
 'offset': 0,
 'totalItemCount': 5,
 'currentLink': 'https://frost.met.no/sources/v0.jsonld?types=SensorSystem&name=Trondheim%2A',
 'data': [{'@type': 'SensorSystem',
   'id': 'SN68230',
   'name': 'TRONDHEIM - RISVOLLAN',
   'shortName': 'Risvollan',
   'country': 'Norge',
   'countryCode': 'NO',
   'geometry': {'@type': 'Point',
    'coordinates': [10.4228, 63.3987],
    'nearest': False},
   'masl': 84,
   'validFrom': '1986-12-11T00:00:00.000Z',
   'county': 'TRØNDELAG',
   'countyId': 50,
   'municipality': 'TRONDHEIM',
   'municipalityId': 5001,
   'stationHolders': ['NVE'],
   'externalIds': ['123.38'],
   'wigosId': '0-578-0-68230'},
  {'@type': 'SensorSystem',
   'id': 'SN68173',
   'name': 'TRONDHEIM - GLØSHAUGEN',
   'sh

In [14]:
json

{'@context': 'https://frost.met.no/schema',
 '@type': 'SourceResponse',
 'apiVersion': 'v0',
 'license': 'https://creativecommons.org/licenses/by/3.0/no/',
 'createdAt': '2019-06-24T05:18:38Z',
 'queryTime': 2.007,
 'currentItemCount': 5,
 'itemsPerPage': 5,
 'offset': 0,
 'totalItemCount': 5,
 'currentLink': 'https://frost.met.no/sources/v0.jsonld?types=SensorSystem&name=Trondheim%2A',
 'data': [{'@type': 'SensorSystem',
   'id': 'SN68230',
   'name': 'TRONDHEIM - RISVOLLAN',
   'shortName': 'Risvollan',
   'country': 'Norge',
   'countryCode': 'NO',
   'geometry': {'@type': 'Point',
    'coordinates': [10.4228, 63.3987],
    'nearest': False},
   'masl': 84,
   'validFrom': '1986-12-11T00:00:00.000Z',
   'county': 'TRØNDELAG',
   'countyId': 50,
   'municipality': 'TRONDHEIM',
   'municipalityId': 5001,
   'stationHolders': ['NVE'],
   'externalIds': ['123.38'],
   'wigosId': '0-578-0-68230'},
  {'@type': 'SensorSystem',
   'id': 'SN68173',
   'name': 'TRONDHEIM - GLØSHAUGEN',
   'sh

## Request available timeSeries from station

In [19]:
station = 'SN68860' # Gløshaugen

In [20]:
src_endpoint = 'https://frost.met.no/observations/availableTimeSeries/v0.jsonld'
parameters = {
    'sources': station,
    'referencetime': '2019-06-01',
}

r = requests.get(src_endpoint, parameters, auth=(client_id,''))
# Extract JSON data
json = r.json()
json

{'@context': 'https://frost.met.no/schema',
 '@type': 'ObservationTimeSeriesResponse',
 'apiVersion': 'v0',
 'license': 'https://creativecommons.org/licenses/by/3.0/no/',
 'createdAt': '2019-06-24T05:23:18Z',
 'queryTime': 0.095,
 'currentItemCount': 136,
 'itemsPerPage': 136,
 'offset': 0,
 'totalItemCount': 136,
 'currentLink': 'https://frost.met.no/observations/availableTimeSeries/v0.jsonld?sources=SN68860&referencetime=2019-06-01',
 'data': [{'sourceId': 'SN68860:0',
   'validFrom': '1951-01-01T00:00:00.000Z',
   'timeOffset': 'PT0H',
   'timeResolution': 'P1D',
   'timeSeriesId': 0,
   'elementId': 'over_time(humidity_mixing_ratio P1D)',
   'unit': 'gr/kg',
   'performanceCategory': 'A',
   'exposureCategory': '1',
   'status': 'Authoritative',
   'uri': 'https://frost.met.noobservations/v0.jsonld?sources=SN68860:0&referencetime=1951-01-01T00:00:00.000Z/9999-12-31T23:59:59Z&elements=over_time(humidity_mixing_ratio P1D)&timeoffsets=PT0H&timeresolutions=P1D&timeseriesids=0&performan

In [55]:
# Get mean air temperature observations for one station for one year

In [103]:
src_endpoint = 'https://frost.met.no/observations/v0.jsonld'
element_list = ['mean(air_temperature P1D)', 'max(air_pressure_at_sea_level P1D)','sum(precipitation_amount P1D)', 'mean(wind_speed P1D)']
parameters = {
    'sources': station,
    'referencetime': '2016-01-01T00:00:00.000Z/2016-12-31T23:59:59Z',
    'elements': ",".join(element_list),
    'timeoffsets': ",".join(['PT0H', 'PT0H', 'PT6H'])
}
#
r = requests.get(src_endpoint, parameters, auth=(client_id,''))
# Extract JSON data
json = r.json()
json

{'@context': 'https://frost.met.no/schema',
 '@type': 'ObservationResponse',
 'apiVersion': 'v0',
 'license': 'https://creativecommons.org/licenses/by/3.0/no/',
 'createdAt': '2019-06-24T06:12:47Z',
 'queryTime': 0.34,
 'currentItemCount': 366,
 'itemsPerPage': 366,
 'offset': 0,
 'totalItemCount': 366,
 'currentLink': 'https://frost.met.no/observations/v0.jsonld?sources=SN68860&referencetime=2016-01-01T00%3A00%3A00.000Z%2F2016-12-31T23%3A59%3A59Z&elements=mean%28air_temperature+P1D%29%2Cmax%28air_pressure_at_sea_level+P1D%29%2Csum%28precipitation_amount+P1D%29%2Cmean%28wind_speed+P1D%29&timeoffsets=PT0H%2CPT0H%2CPT6H',
 'data': [{'sourceId': 'SN68860:0',
   'referenceTime': '2016-01-01T00:00:00.000Z',
   'observations': [{'elementId': 'mean(air_temperature P1D)',
     'value': 4.3,
     'unit': 'degC',
     'level': {'levelType': 'height_above_ground', 'unit': 'm', 'value': 2},
     'timeOffset': 'PT0H',
     'timeResolution': 'P1D',
     'timeSeriesId': 0,
     'performanceCategory':

## Read into pandas DataFrame

In [ ]:
df_weather = pd.DataFrame.from_records(json["data"])

In [99]:
# Extract values
for i,e in enumerate(df_weather["observations"][0]):
    df_weather[e['elementId']+"_"+e['timeOffset']] = df_weather["observations"].apply(lambda x: x[i]["value"])

In [100]:
df_weather.head()

,observations,referenceTime,sourceId,mean(air_temperature P1D)_PT0H,mean(air_temperature P1D)_PT6H,max(air_pressure_at_sea_level P1D)_PT0H,sum(precipitation_amount P1D)_PT6H
0,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-01-01T00:00:00.000Z,SN68860:0,0.9,-2.9,986.3,0.0
1,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-01-02T00:00:00.000Z,SN68860:0,0.1,0.8,996.2,0.7
2,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-01-03T00:00:00.000Z,SN68860:0,-3.3,-1.4,996.1,0.2
3,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-01-04T00:00:00.000Z,SN68860:0,-1.6,-2.5,999.3,0.0
4,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-01-05T00:00:00.000Z,SN68860:0,-3.4,-1.7,1003.6,0.0


In [101]:
df_weather.head(20)

,observations,referenceTime,sourceId,mean(air_temperature P1D)_PT0H,mean(air_temperature P1D)_PT6H,max(air_pressure_at_sea_level P1D)_PT0H,sum(precipitation_amount P1D)_PT6H
0,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-01-01T00:00:00.000Z,SN68860:0,0.9,-2.9,986.3,0.0
1,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-01-02T00:00:00.000Z,SN68860:0,0.1,0.8,996.2,0.7
2,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-01-03T00:00:00.000Z,SN68860:0,-3.3,-1.4,996.1,0.2
3,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-01-04T00:00:00.000Z,SN68860:0,-1.6,-2.5,999.3,0.0
4,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-01-05T00:00:00.000Z,SN68860:0,-3.4,-1.7,1003.6,0.0
5,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-01-06T00:00:00.000Z,SN68860:0,-3.4,-3.9,1017.4,0.0
6,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-01-07T00:00:00.000Z,SN68860:0,-1.1,-3.0,1022.8,5.7
7,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-01-08T00:00:00.000Z,SN68860:0,0.9,-0.4,1027.5,6.1
8,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-01-09T00:00:00.000Z,SN68860:0,-2.4,0.7,1027.6,2.0
9,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-01-10T00:00:00.000Z,SN68860:0,-4.0,-3.6,1021.5,0.0


In [104]:
dfs = []
for year in [str(y) for y in range(2016,2019)]:
    src_endpoint = 'https://frost.met.no/observations/v0.jsonld'
    #element_list = ['mean(air_temperature P1D)', 'max(air_pressure_at_sea_level P1D)','sum(precipitation_amount P1D)']
    parameters = {
        'sources': station,
        'referencetime': f'{year}-01-01T00:00:00.000Z/{year}-12-31T23:59:59Z',
        'elements': ",".join(element_list),
        'timeoffsets': ",".join(['PT0H', 'PT0H', 'PT6H'])
    }
    #
    r = requests.get(src_endpoint, parameters, auth=(client_id,''))
    # Extract JSON data
    json = r.json()
    json
    df_weather = pd.DataFrame.from_records(json["data"])
    # Extract values
    for i,e in enumerate(df_weather["observations"][0]):
        df_weather[e['elementId']+"_"+e['timeOffset']] = df_weather["observations"].apply(lambda x: x[i]["value"])
    dfs.append(df_weather)

In [105]:
df_tot = pd.concat(dfs)

In [107]:
df_tot.shape

(1096, 8)

In [108]:
df_tot.sample(10)

,observations,referenceTime,sourceId,mean(air_temperature P1D)_PT0H,mean(air_temperature P1D)_PT6H,max(air_pressure_at_sea_level P1D)_PT0H,sum(precipitation_amount P1D)_PT6H,mean(wind_speed P1D)_PT0H
292,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2017-10-20T00:00:00.000Z,SN68860:0,5.9,3.2,1018.3,0.1,2.3
361,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-12-28T00:00:00.000Z,SN68860:0,3.9,4.6,1016.4,2.5,2.9
341,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2016-12-07T00:00:00.000Z,SN68860:0,0.5,0.8,1020.2,0.1,1.5
228,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2016-08-16T00:00:00.000Z,SN68860:0,13.3,11.2,1025.6,0.2,1.4
184,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-07-04T00:00:00.000Z,SN68860:0,13.2,13.0,1018.9,0.1,2.7
88,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2016-03-29T00:00:00.000Z,SN68860:0,7.2,8.1,997.7,0.0,4.1
317,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-11-14T00:00:00.000Z,SN68860:0,2.7,2.8,1015.8,0.0,1.0
196,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2018-07-16T00:00:00.000Z,SN68860:0,20.7,18.9,1017.2,0.0,2.1
163,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2016-06-12T00:00:00.000Z,SN68860:0,10.5,11.5,1016.9,0.0,2.2
132,"[{'elementId': 'mean(air_temperature P1D)', 'v...",2017-05-13T00:00:00.000Z,SN68860:0,10.2,8.6,1015.2,0.0,2.1
